In [ ]:
source extend_bashkernel-2modes.source
set-global-evaluation-mode "my-script"

Ready.


In [ ]:
set-global-evaluation-mode "my-script"

# YUM リポジトリ

## はじめに

yumリポジトリ作成手順を学ぶには、最小構成のyumリポジトリを作ってみる事が一番の近道です。本章では、実際に最小構成yumリポジトリを作りながらyumリポジトリ構築の基礎を学んで行きます。

## 前提条件

本書は「RPM 生成方法」の続編とも言うべき内容で、example-0.1.0.rpmが生成済みである事を前提に説明して行きます。

## 演習について

* コマンドは全てsshを利用してJenkinsが稼働するマシンで実行します。
* スクリプト例に記載されているコードは実際に実行できます。課題をどう解決したら良いか解らなくなったら参考にして下さい。

## 何故Yumなのか?

rpmファイルを効率良く管理するには、yumの力を借りる事です。それは何故でしょうか？主に3つの理由があります。

1. 依存関係の解決
2. rpmファイル配置場所の意識
3. バージョンの解決

それぞれの理由を少し掘り下げてみます。

### 理由1: 依存関係の解決

rpmspecファイルには依存関係を定義出来ますが、rpmコマンドで依存パッケージのインストールまでは面倒見てくれません。それに対し、yumコマンドは依存パッケージのインストールまで面倒見てくれます。

### 理由2: rpmファイル配置場所の意識

rpmコマンドでrpmファイルをインストールする場合、rpmファイルの場所を意識する必要があります。以下に例を示します。

```
$ sudo rpm -ivh /var/www/html/pub/example-0.1.0-1.el6.x86_64.rpm
$ sudo rpm -ivh http://ftp.example.com/pub/example-0.1.0-1.el6.x86_64.rpm
$ sudo rpm -ivh ftp://ftp.example.com/pub/example-0.1.0-1.el6.x86_64.rpm
```

yumコマンドの場合は、yumリポジトリを事前設定しておく事により、rpmファイルの場所を意識する事無くインストールが可能です。

```
$ sudo yum install -y example
```

パッケージ数が1つや2つ程度であれば大した問題にはなりませんが、数十から数百も扱う場合は、管理運用コストが高くなります。

### 理由3: バージョン情報

yumリポジトリ内に複数バージョンのパッケージが存在する場合も上手に扱ってくれます。仮に`example-0.1.0`と`example-0.2.0`が存在しているとします。バージョンを意識したインストールの場合は、`パッケージ名-バージョン`でインストール可能です。

```
$ sudo yum install -y example-0.1.0
$ sudo yum install -y example-0.2.0
```

また、`パッケージ名`のみ指定する事も可能です。

```
$ sudo yum install -y example
```

さて、この場合、どのバージョンがインストールされるでしょうか？答えは、`example-0.2.0`です。yumはリポジトリ内にどのバージョンが最新であるかを判断してインストールを行います。この仕組みは、新しいパッケージを配布したい時に有効な手段です。同じインストールコマンドを使い、常に最新バージョンをインストール可能です。インストール手順を定義する際にも役立ちます。

## ここまでのまとめ

3つの理由を踏まえると、もはやyumを利用しない理由が見当たりません。

# Yumリポジトリ公開に必要なもの

Yumリポジトリを公開するには、3つの要素が必要です。

1. 公開するrpmファイル
2. Yumリポジトリ
3. Yumリポジトリ用repoファイル

# 環境構築

## 演習 : 必要なパッケージをインストールする

### 演習の内容

Yumリポジトリを構築するには`createrepo`が必要です。また、YumリポジトリをHTTPで公開する為に、`httpd`をインストールします。yumはFTPもサポートしているのでFTPでも構いませんが、本書ではHTTPで公開する事を前提に進めます。
```
$ sudo yum install -y createrepo httpd
```
実行結果例：

> ```
> $ sudo yum install -y createrepo httpd
> Loaded plugins: fastestmirror
> Setting up Install Process
> Determining fastest mirrors
>  * base: ftp.riken.jp
>  * extras: ftp.riken.jp
>  * updates: ftp.riken.jp
> ...(省略)...
> Complete!
> ```

**次のセルでパッケージのインストールを実行して下さい。**

### スクリプト例

In [ ]:
# Task: yum-host-install-packages
# Evaluation Mode: provided-script

# Install packages

ssh -qi ../mykeypair root@10.0.2.100 'sudo yum install -y createrepo httpd'

### 確認
次のセルを実行して、必要なパッケージのインストールが完了していることを確認して下さい。

In [ ]:
# Task: yum-host-install-packages
# Evaluation Mode: check

## 演習 : パッケージ公開用HTTPDをセットアップする

### 演習の内容

パッケージ公開用のhttpdを起動して、自動起動を有効化します。

**httpdを起動する**

yumコマンドによるhttpdパッケージのインストールでは、サービス起動ませんので、httpdサービスを起動します。
```
$ sudo service httpd start
```

実行結果例：
> ```
> $ sudo service httpd start
> Starting httpd: httpd: apr_sockaddr_info_get() failed for vagrant-centos6
> httpd: Could not reliably determine the server's fully qualified domain name, using 127.0.0.1 for ServerName
>                                                            [  OK  ]
> ```

**自動起動の有効化**

パッケージインストール直後の初期設定では、システム起動時の自動起動が無効化されています。今回の用途においては、システム起動時に自動起動して欲しいので、自動起動を有効化します。

* 自動起動有効化前の内容確認

  設定変更前の自動起動設定を確認してみます。
```
$ chkconfig --list httpd
```

  実行結果例：
> ```
> $ chkconfig --list httpd
> httpd           0:off   1:off   2:off   3:off   4:off   5:off   6:off
> ```

  `3:off`となっており、自動起動が無効化されています。


* 自動起動の有効化

  自動起動を有効化します。
```
$ sudo chkconfig httpd on
```

  実行結果例：
> ```
> $ sudo chkconfig httpd on
> ```

* 自動起動有効化の確認

  自動起動が有効化されてる事を確認します。
```
$ chkconfig --list httpd
```

  実行結果例：
> ```
> $ chkconfig --list httpd
> httpd           0:off   1:off   2:on    3:on    4:on    5:on    6:off
> ```

  `3:on`となっており、自動起動が有効化されました。これで次回のシステム起動から、httpdが自動起動します。


**次のセルで、httpdの起動と自動起動の設定を実行して下さい。**

**[ヒント]** リモートで複数のコマンドを実行するには、ヒアドキュメントを利用します。

### スクリプト例

In [ ]:
# Task: yum-host-setup-httpd
# Evaluation Mode: provided-script

# Configure and confirm/reconfirm httpd status

ssh -qi ../mykeypair root@10.0.2.100 <<'EOF'
sudo service httpd start
sudo chkconfig --list httpd
sudo chkconfig httpd on
sudo chkconfig --list httpd
EOF

### 確認
次のセルを実行して、httpdのセットアップが完了していることを確認して下さい。

In [ ]:
# Task: yum-host-setup-httpd
# Evaluation Mode: check

# 独自Yumリポジトリの構築

## 演習 : 公開用ディレクトリを準備する

### 演習の内容

Yumリポジトリ公開用ディレクトリを準備します。本書では`/pub/`を公開用ディレクトリとして使用します。

```
$ sudo mkdir -p /var/www/html/pub
```

実行結果例：
> ```
> $ sudo mkdir -p /var/www/html/pub
> ```

```
$ ls -l /var/www/html/pub/
```

実行結果例：
> ```
> $ ls -l /var/www/html/pub/
> total 0
> ```

**次のセルで、公開用ディレクトリの作成を実行して下さい。**

### スクリプト例

In [ ]:
# Task: yum-host-setup-publish-folder
# Evaluation Mode: provided-script

# Cretae the public folder httpd server and confirm

ssh -qi ../mykeypair root@10.0.2.100 <<'EOF'
sudo mkdir -p /var/www/html/pub
sudo ls -la /var/www/html/pub
EOF

### 確認
次のセルを実行して、公開用ディレクトリが作成されていることを確認して下さい。

In [ ]:
# Task: yum-host-setup-publish-folder
# Evaluation Mode: check

## 演習 : rpmファイルを確認する

### 演習の内容

事前にyumリポジトリに登録するrpmファイルを確認しておきます。「RPM 生成方法」で生成した`example-0.1.0-1.el6.x86_64.rpm`があるはずです。

```
$ ls -l ${HOME}/rpmbuild/RPMS/x86_64/example-0.1.0-1.el6.x86_64.rpm
```

> ```
> $ ls -l ${HOME}/rpmbuild/RPMS/x86_64/example-0.1.0-1.el6.x86_64.rpm
> -rw-r--r-- 1 vagrant vagrant 1767 Apr 15 19:20 /home/vagrant/rpmbuild/RPMS/x86_64/example-0.1.0-1.el6.x86_64.rpm
> ```

**次のセルで、rpmファイル存在確認を実行して下さい。**

### スクリプト例

In [ ]:
# Task: yum-host-confirm-rpm
# Evaluation Mode: provided-script

# Confirm that the rpm exists

ssh -qi ../mykeypair root@10.0.2.100 ' ls -l ${HOME}/rpmbuild/RPMS/x86_64/example-0.1.0-1.el6.x86_64.rpm'

### 確認
次のセルを実行して、公開用ディレクトリ作成が完了していることを確認して下さい。

In [ ]:
# Task: yum-host-confirm-rpm
# Evaluation Mode: check

### 演習 : rpmファイルを配置する

### 演習の内容

**rpmファイルを公開用ディレクトリに配置します**

配置作業はrsyncで行います。
```
$ cd ${HOME}/rpmbuild/RPMS/
$ sudo rsync -avx . /var/www/html/pub/
```

実行結果例：
> ```
> $ sudo rsync -avx . /var/www/html/pub/
> sending incremental file list
> ./
> x86_64/
> x86_64/example-0.1.0-1.el6.x86_64.rpm
>
> sent 1925 bytes  received 38 bytes  3926.00 bytes/sec
> total size is 1767  speedup is 0.90
> ```


**rpmファイルが配置された事を確認します**
```
$ ls -l /var/www/html/pub/x86_64/example-0.1.0-1.el6.x86_64.rpm
```

実行結果例：
> ```
> $ ls -l /var/www/html/pub/x86_64/example-0.1.0-1.el6.x86_64.rpm
> -rw-r--r-- 1 vagrant vagrant 1767 Apr 15 19:20 /var/www/html/pub/x86_64/example-0.1.0-1.el6.x86_64.rpm
> ```


**次のセルで、上記のrpmファイルの配置手順を実行して下さい。**

### スクリプト例

In [ ]:
# Task: yum-host-publish-rpm
# Evaluation Mode: provided-script

# Copy the rpm to newly created published folder

ssh -qi ../mykeypair root@10.0.2.100 <<'EOF'
cd ${HOME}/rpmbuild/RPMS/
sudo rsync -avx . /var/www/html/pub/
ls -l /var/www/html/pub/x86_64/example-0.1.0-1.el6.x86_64.rpm
EOF

### 確認
次のセルを実行して、rpmファイルの配置が完了していることを確認して下さい。

In [ ]:
# Task: yum-host-publish-rpm
# Evaluation Mode: check

## 演習 : yumリポジトリを構築する

### 演習の内容

**ディレクトリを移動します**

`createrepo`コマンドを実行する前に、ディレクトリを移動しておく必要があります。
```
$ cd /var/www/html/pub
```

**`createrepo`コマンドを実行します**

`createrepo`コマンドを実行します。
```
$ sudo createrepo .
```

実行結果例：
> ```
> $ sudo createrepo .
> Spawning worker 0 with 1 pkgs
> Workers Finished
> Gathering worker results
>
> Saving Primary metadata
> Saving file lists metadata
> Saving other metadata
> Generating sqlite DBs
> Sqlite DBs complete
> ```

実行すると、`repodata`ディレクトリが生成されます。
```
$ ls -l repodata/
```

実行結果例：
> ```
> total 36
> drwxr-xr-x 2 root    root    4096 Apr 27 15:44 .
> drwxr-xr-x 4 vagrant vagrant 4096 Apr 27 15:44 ..
> -rw-r--r-- 1 root    root     232 Apr 27 15:44 21f3ad602df029deeb143d8c75df67e18c0ce2036e76e9e94d42479f6d7302a5-other.xml.gz
> -rw-r--r-- 1 root    root     268 Apr 27 15:44 2c14a8b954d727e5106e5646f99cf3ab65d1ceaff77173bc4038107fc3c791b1-filelists.xml.gz
> -rw-r--r-- 1 root    root     748 Apr 27 15:44 711c22507ef9815a06e9f9bf888f3468c9b98f1fa1f65aed1ff78ae27e0d6c62-filelists.sqlite.bz2
> -rw-r--r-- 1 root    root    1632 Apr 27 15:44 8754285698aa24ef37fb9d7e8b88d3f6ad4a8c267da420626b6e6d9ba4a8a2d0-primary.sqlite.bz2
> -rw-r--r-- 1 root    root     577 Apr 27 15:44 8ec3972ca9194bc3b1e759900dc3d6e69334bd5b98281fd7d93ed7f1c97a6289-primary.xml.gz
> -rw-r--r-- 1 root    root     650 Apr 27 15:44 d3c30d5e41a3e637ad5493488c5f1278f131e2d75cdc09ee480a969ed2ed9625-other.sqlite.bz2
> -rw-r--r-- 1 root    root    2972 Apr 27 15:44 repomd.xml
> ```

Yumリポジトリが作成されました。

**次のセルで、Yumリポジトリの作成を実行して下さい。**

### スクリプト例

In [ ]:
# Task: yum-host-publish-repo
# Evaluation Mode: provided-script

# Create the repo

ssh -qi ../mykeypair root@10.0.2.100 <<'EOF'
cd /var/www/html/pub
sudo createrepo .
ls -l repodata/
EOF

### 確認
次のセルを実行して、Yumリポジトリの作成が完了していることを確認して下さい。

In [ ]:
# Task: yum-host-publish-repo
# Evaluation Mode: check

# repoファイルの作成

## 演習 : repoファイルを配置する

### 演習の内容

次のrepoファイルを、/etc/yum.repos.d/example.repoに配置します。
なお、`baseurl`の`127.0.0.1`は、環境に合わせて指定して下さい。

```
[example]
name=example
baseurl=http://127.0.0.1/pub/
enabled=1
gpgcheck=0
```

**次のセルで、repoファイルの配置を実行して下さい。**

**[ヒント]** リモートでのファイル作成には、ヒアドキュメントを利用します。

### スクリプト例

In [ ]:
# Task: yum-host-add-repo
# Evaluation Mode: provided-script

# Add the repo to list

ssh -qi ../mykeypair root@10.0.2.100 <<'EOS'
cat <<EOF > /etc/yum.repos.d/example.repo
[example]
name=example
baseurl=http://127.0.0.1/pub/
enabled=1
gpgcheck=0
EOF
EOS

### 確認
次のセルを実行して、repoファイルの配置が完了していることを確認して下さい。

In [ ]:
# Task: yum-host-add-repo
# Evaluation Mode: check

## 演習 : Yumリポジトリを使ってインストールする

### 演習の内容

**リポジトリが有効かどうかを確認します**

リポジトリが利用可能かどうかを確認します。

```
$ sudo yum makecache --disablerepo='*' --enablerepo='example'
```

実行結果例：

> ```
> $ sudo yum makecache --disablerepo='*' --enablerepo='example'
> Loaded plugins: fastestmirror
> Loading mirror speeds from cached hostfile
> example                                                  | 2.9 kB     00:00
> example/filelists_db                                     |  748 B     00:00
> example/other_db                                         |  650 B     00:00
> Metadata Cache Created
> ```

`example`が有効である事が分かります。

**`example`パッケージ情報を取得します**

Yumリポジトリが利用可能になったので、`example`パッケージ情報を取得できるはずです。
```
$ yum info example
```

実行結果例：
> ```
> $ yum info example
> Loaded plugins: fastestmirror
> Determining fastest mirrors
>  * base: www.ftp.ne.jp
>  * extras: www.ftp.ne.jp
>  * updates: www.ftp.ne.jp
> Available Packages
> Name        : example
> Arch        : x86_64
> Version     : 0.1.0
> Release     : 1.el6
> Size        : 1.7 k
> Repo        : example
> Summary     : example
> License     : BSD
> Description :
> ```


**Yumリポジトリを使ってインストールしてみる**

yumコマンドで`example`パッケージをインストールしてみます。
```
$ sudo yum install -y example
```

実行結果例：
> ```
> $ sudo yum install -y example
> Loaded plugins: fastestmirror
> Setting up Install Process
> Loading mirror speeds from cached hostfile
>  * base: ftp.iij.ad.jp
>  * extras: centos.usonyx.net
>  * updates: centos.usonyx.net
> Resolving Dependencies
> --> Running transaction check
> ---> Package example.x86_64 0:0.1.0-1.el6 will be installed
> --> Finished Dependency Resolution
>
> Dependencies Resolved
>
> ================================================================================
>  Package          Arch            Version                Repository        Size
> ================================================================================
> Installing:
>  example          x86_64          0.1.0-1.el6            example          1.7 k
>
> Transaction Summary
> ================================================================================
> Install       1 Package(s)
>
> Total download size: 1.7 k
> Installed size: 0
> Downloading Packages:
> example-0.1.0-1.el6.x86_64.rpm                           | 1.7 kB     00:00
> Running rpm_check_debug
> Running Transaction Test
> Transaction Test Succeeded
> Running Transaction
>   Installing : example-0.1.0-1.el6.x86_64                                   1/1
>   Verifying  : example-0.1.0-1.el6.x86_64                                   1/1
>
> Installed:
>   example.x86_64 0:0.1.0-1.el6
>
> Complete!
>
> ```


**次のセルで、上記のYumリポジトリを使ったパッケージインストール手順を実行して下さい。**

### スクリプト例

In [ ]:
# Task: yum-host-install-example-package
# Evaluation Mode: provided-script

# Check information and install the example package

ssh -qi ../mykeypair root@10.0.2.100 <<'EOF'
sudo yum makecache --disablerepo='*' --enablerepo='example'
sudo yum info example
sudo yum install -y example
EOF

### 確認
次のセルを実行して、Yumリポジトリを使ったパッケージインストール手順が完了していることを確認して下さい。

In [ ]:
# Task: yum-host-install-example-package
# Evaluation Mode: check

# 複数バージョンrpmファイルの運用管理

## 演習 : example-0.2.0のアーカイブファイルを作成する

### 演習の内容

**example-0.2.0を作成します**

SOURCEディレクトリに移動して、example-0.1.0を変更し、example-0.2.0を作成します。
```
$ cd ${HOME}/rpmbuild/SOURCES/
$ rsync -avx example-0.1.0/ example-0.2.0
```

実行結果例：

> ```
> $ rsync -avx example-0.1.0/ example-0.2.0
> sending incremental file list
> created directory example-0.2.0
> ./
> bin/
> bin/example
>
> sent 128 bytes  received 38 bytes  332.00 bytes/sec
> total size is 0  speedup is 0.00
> ```


**アーカイブファイルを作成します**

```
$ tar zcvf example-0.2.0.tar.gz example-0.2.0
```

実行結果例：

> ```
> $ tar zcvf example-0.2.0.tar.gz example-0.2.0
> example-0.2.0/
> example-0.2.0/bin/
> example-0.2.0/bin/example
> ```


**次のセルで、上記のアーカイブファイル作成手順を実行して下さい。**

### スクリプト例

In [ ]:
# Task: yum-host-upgrade-version
# Evaluation Mode: provided-script

# Adding a new version

ssh -qi ../mykeypair root@10.0.2.100 <<'EOF'
cd ${HOME}/rpmbuild/SOURCES
rsync -avx example-0.1.0/ example-0.2.0
tar zcvf example-0.2.0.tar.gz example-0.2.0
EOF

### 確認
次のセルを実行して、アーカイブファイル作成手順が完了していることを確認して下さい。

In [ ]:
# Task: yum-host-upgrade-version
# Evaluation Mode: check

## 演習 : rpmspecファイルを更新する

### 演習の内容

**rpmspecファイルを修正します**

```
$ cd ${HOME}/rpmbuild/SPECS/
```

作業前後の差分を確認する為に、rpmspecファイルのバックアップを作成します。

```
$ cp -pi example.spec example.spec.orig
```

**バージョン番号の更新**

バージョンを`0.1.0`から`0.2.0`へ進めます。

```
$ sed -i 's,0.1.0,0.2.0,' example.spec
```

**更新を確認します**

作業前後の差分を確認します。

```
$ diff example.spec.orig example.spec
```

実行結果例：

> ```
> $ diff example.spec.orig example.spec
. 2c2
> < Version: 0.1.0
> ---
> > Version: 0.2.0
> ```


**次のセルで、rpmspecファイルの修正手順を実行して下さい。**

### スクリプト例

In [ ]:
# Task: yum-host-upgrade-spec
# Evaluation Mode: provided-script

# Upgrade the information inside spec file

ssh -qi ../mykeypair root@10.0.2.100 <<'EOF'
cd ${HOME}/rpmbuild/SPECS/
cp -pi example.spec example.spec.orig
sed -i 's,0.1.0,0.2.0,' example.spec
diff example.spec.orig example.spec
EOF

### 確認
次のセルを実行して、rpmspecファイルの修正が完了していることを確認して下さい。

In [ ]:
# Task: yum-host-upgrade-spec
# Evaluation Mode: check

## 演習 : example-0.2.0のrpmをビルドする

### 演習の内容

**rpmをビルドします**

```
$ rpmbuild -bb example.spec
```

実行結果例：

> ```
> $ rpmbuild -bb example.spec
> Executing(%prep): /bin/sh -e /var/tmp/rpm-tmp.R28MbM
> + umask 022
> + cd /home/vagrant/rpmbuild/BUILD
> + LANG=C
> + export LANG
> + unset DISPLAY
> + cd /home/vagrant/rpmbuild/BUILD
> + rm -rf example-0.2.0
> + /bin/tar -xf -
> + /usr/bin/gzip -dc /home/vagrant/rpmbuild/SOURCES/example-0.2.0.tar.gz
> + STATUS=0
> + '[' 0 -ne 0 ']'
> + cd example-0.2.0
> + /bin/chmod -Rf a+rX,u+w,g-w,o-w .
> + exit 0
> Executing(%build): /bin/sh -e /var/tmp/rpm-tmp.kM1DG9
> + umask 022
> + cd /home/vagrant/rpmbuild/BUILD
> + cd example-0.2.0
> + LANG=C
> + export LANG
> + unset DISPLAY
> + exit 0
> Executing(%install): /bin/sh -e /var/tmp/rpm-tmp.iqJMbx
> + umask 022
> + cd /home/vagrant/rpmbuild/BUILD
> + '[' /home/vagrant/rpmbuild/BUILDROOT/example-0.2.0-1.el6.x86_64 '!=' / ']'
> + rm -rf /home/vagrant/rpmbuild/BUILDROOT/example-0.2.0-1.el6.x86_64
> ++ dirname /home/vagrant/rpmbuild/BUILDROOT/example-0.2.0-1.el6.x86_64
> + mkdir -p /home/vagrant/rpmbuild/BUILDROOT
> + mkdir /home/vagrant/rpmbuild/BUILDROOT/example-0.2.0-1.el6.x86_64
> + cd example-0.2.0
> + LANG=C
> + export LANG
> + unset DISPLAY
> + rm -rf /home/vagrant/rpmbuild/BUILDROOT/example-0.2.0-1.el6.x86_64
> + mkdir -p /home/vagrant/rpmbuild/BUILDROOT/example-0.2.0-1.el6.x86_64
> + rsync -avx ./ /home/vagrant/rpmbuild/BUILDROOT/example-0.2.0-1.el6.x86_64/
> sending incremental file list
> ./
> bin/
> bin/example
>
> sent 128 bytes  received 38 bytes  332.00 bytes/sec
> total size is 0  speedup is 0.00
> + /usr/lib/rpm/find-debuginfo.sh --strict-build-id /home/vagrant/rpmbuild/BUILD/example-0.2.0
> + /usr/lib/rpm/check-buildroot
> + /usr/lib/rpm/redhat/brp-compress
> + /usr/lib/rpm/redhat/brp-strip-static-archive /usr/bin/strip
> + /usr/lib/rpm/redhat/brp-strip-comment-note /usr/bin/strip /usr/bin/objdump
> + /usr/lib/rpm/brp-python-bytecompile
> + /usr/lib/rpm/redhat/brp-python-hardlink
> + /usr/lib/rpm/redhat/brp-java-repack-jars
> Processing files: example-0.2.0-1.el6.x86_64
> Requires(rpmlib): rpmlib(CompressedFileNames) <= 3.0.4-1 rpmlib(FileDigests) <= 4.6.0-1 rpmlib(PayloadFilesHavePrefix) <= 4.0-1
> Processing files: example-debuginfo-0.2.0-1.el6.x86_64
> Checking for unpackaged file(s): /usr/lib/rpm/check-files /home/vagrant/rpmbuild/BUILDROOT/example-0.2.0-1.el6.x86_64
> warning: Could not canonicalize hostname: vagrant-centos6
> Wrote: /home/vagrant/rpmbuild/RPMS/x86_64/example-0.2.0-1.el6.x86_64.rpm
> Wrote: /home/vagrant/rpmbuild/RPMS/x86_64/example-debuginfo-0.2.0-1.el6.x86_64.rpm
> Executing(%clean): /bin/sh -e /var/tmp/rpm-tmp.CnU3m2
> + umask 022
> + cd /home/vagrant/rpmbuild/BUILD
> + cd example-0.2.0
> + rm -rf /home/vagrant/rpmbuild/BUILDROOT/example-0.2.0-1.el6.x86_64
> + exit 0
> ```

**次のセルで、rpmビルドを実行して下さい。**

### スクリプト例

In [ ]:
# Task: yum-host-upgrade-rebuild-spec
# Evaluation Mode: provided-script

# Rebuild the spec file to be versoin 0.2.0

ssh -qi ../mykeypair root@10.0.2.100 <<'EOF'
cd ${HOME}/rpmbuild/SPECS/
rpmbuild -bb example.spec
EOF

### 確認
次のセルを実行して、rpmビルドが完了していることを確認して下さい。

In [ ]:
# Task: yum-host-upgrade-rebuild-spec
# Evaluation Mode: check

## 演習 : rpmを/var/www/html/pub/に配置する

### 演習の内容

example-0.2.0を`/var/www/html/pub/`に配置して行きます。

```
$ cd ${HOME}/rpmbuild/RPMS/
```

```
$ sudo rsync -avx . /var/www/html/pub/
```

実行結果例：

> ```
> $ sudo rsync -avx . /var/www/html/pub/
> sending incremental file list
> ./
> x86_64/
> x86_64/example-0.2.0-1.el6.x86_64.rpm
> x86_64/example-debuginfo-0.2.0-1.el6.x86_64.rpm
>
> sent 3929 bytes  received 57 bytes  7972.00 bytes/sec
> total size is 5427  speedup is 1.36
> ```

配置先にrpmファイルがある事を確認します。

```
$ ls -l /var/www/html/pub/x86_64/example-0.2.0-1.el6.x86_64.rpm
```

実行結果例：

> ```
> $ ls -l /var/www/html/pub/x86_64/example-0.2.0-1.el6.x86_64.rpm
> -rw-r--r-- 1 vagrant vagrant 2064 Apr 27 20:12 /var/www/html/pub/x86_64/example-0.2.0-1.el6.x86_64.rpm
> ```

**次のセルで、rpmファイルの配置を実行して下さい。**

### スクリプト名

In [ ]:
# Task: yum-host-upgrade-republish-spec
# Evaluation Mode: provided-script

# Publish new version on the webserver

ssh -qi ../mykeypair root@10.0.2.100 <<'EOF'
cd ${HOME}/rpmbuild/RPMS/
sudo rsync -avx . /var/www/html/pub
ls -l /var/www/html/pub/x86_64/example-0.2.0-1.el6.x86_64.
EOF

### 確認
次のセルを実行して、rpmの配置が完了していることを確認して下さい。

In [ ]:
# Task: yum-host-upgrade-republish-spec
# Evaluation Mode: check

## 演習 : example-0.2.0を含めたYumリポジトリを構築する

### 演習の内容

example-0.2.0を含めたYumリポジトリを構築して行きます。

```
$ cd /var/www/html/pub
```

```
$ sudo createrepo .
```

実行結果例：

> ```
> $ sudo createrepo .
> Spawning worker 0 with 3 pkgs
> Workers Finished
> Gathering worker results
>
> Saving Primary metadata
> Saving file lists metadata
> Saving other metadata
> Generating sqlite DBs
> Sqlite DBs complete
> ```

**次のセルで、Yumリポジトリの構築を実行して下さい。**

### スクリプト例

In [ ]:
# Task: yum-host-upgrade-publish-repo
# Evaluation Mode: provided-script

ssh -qi ../mykeypair root@10.0.2.100 <<'EOF'
cd /var/www/html/pub
sudo createrepo .
EOF

### 確認
次のセルを実行して、Yamリポジトリの構築が完了していることを確認して下さい。

In [ ]:
# Task: yum-host-upgrade-publish-repo
# Evaluation Mode: check

## 演習 : 新exampleパッケージが存在するか確認する

### 演習の内容

新Yumリポジトリに新exampleパッケージが存在するかどうかを確認します。

```
$ yum clean metadata
$ yum info example
```

実行結果例：

> ```
> $ yum clean metadata
> ...(省略)...
> $ yum info example
> Loaded plugins: fastestmirror
> Loading mirror speeds from cached hostfile
>  * base: www.ftp.ne.jp
>  * extras: www.ftp.ne.jp
>  * updates: www.ftp.ne.jp
> Installed Packages
> Name        : example
> Arch        : x86_64
> Version     : 0.1.0
> Release     : 1.el6
> Size        : 0.0
> Repo        : installed
> From repo   : example
> Summary     : example
> License     : BSD
> Description :
>
> Available Packages
> Name        : example
> Arch        : x86_64
> Version     : 0.2.0
> Release     : 1.el6
> Size        : 2.0 k
> Repo        : example
> Summary     : example
> License     : BSD
> Description :
>
> ```

`0.2.0`が最新版である事が分かります。

**次のセルで、新exampleパッケージの存在確認を実行して下さい。**

### スクリプト例

In [ ]:
# Task: yum-host-upgrade-reset-metadata
# Evaluation Mode: provided-script

ssh -qi ../mykeypair root@10.0.2.100 <<'EOF'
yum clean metadata
yum info example
EOF

### 確認
次のセルを実行して、新パッケージの存在確認手順が完了していることを確認して下さい。

In [ ]:
# Task: yum-host-upgrade-reset-metadata
# Evaluation Mode: check

## 演習 : パッケージをアップデートしてみる

### 演習の内容

インストールされたパッケージの情報を確認してみます。

```
$ rpm -qi example
```

実行結果例：

> ```
> $ rpm -qi example
> Name        : example                      Relocations: (not relocatable)
> Version     : 0.2.0                             Vendor: (none)
> Release     : 1.el6                         Build Date: Mon 27 Apr 2015 08:12:16 PM JST
> Install Date: Mon 27 Apr 2015 08:23:15 PM JST      Build Host: vagrant-centos6
> Group       : Unspecified                   Source RPM: example-0.2.0-1.el6.src.rpm
> Size        : 0                                License: BSD
> Signature   : (none)
> Summary     : example
> Description :
> ```

example-0.2.0がインストールされた事を確認出来ました。

**次のセルで、上記のパッケージアップデート手順を実行して下さい。**

### スクリプト例

In [ ]:
# Task: yum-host-upgrade-install
# Evaluation Mode: provided-script

ssh -qi ../mykeypair root@10.0.2.100 <<'EOF'
sudo yum update -y example
rpm -qi example
EOF

### 確認
次のセルを実行して、パッケージアップデート手順が完了していることを確認して下さい。

In [ ]:
# Task: yum-host-upgrade-install
# Evaluation Mode: check

## ここまでのまとめ

複数バージョンのrpmが混在する場合のYumリポジトリ構築を学びました。これ以降は、パッケージ内容を変更する度に、同じ手順で更新して行くだけです。手順を上手に整理すると、一連の作業を自動化可能です。

# 複数台構成を意識した手順整理

これまでに最小構成Yumリポジトリを作ってYumリポジトリ構築と更新を学びました。1台構成でしたが、通常は複数台構成です。複数台構成を意識した手順で整理します。

役割：

1. rpmビルドサーバ/Yumリポジトリ
2. アプリケーションサーバ(パッケージインストール実施環境)

それでは役割毎に手順を整理してみます。

## rpmビルドサーバ / Yumリポジトリ

### rpmのビルド

```
$ cd ${HOME}/rpmbuild/SPECS/
$ rpmbuild -bb example.spec
```

### RPMSの配置

```
$ cd ${HOME}/rpmbuild/RPMS/
$ sudo rsync -avx . /var/www/html/pub/
```

### Yumリポジトリの構築

```
$ cd /var/www/html/pub
$ sudo createrepo .
```

## アプリケーションサーバ

### repoファイルの配置

baseurlに指定するIPアドレスを、ビルドサーバのIPアドレスにします。
```
$ sudo vi /etc/yum.repos.d/example.repo
```

### パッケージのインストール/更新

初期インストールの場合：

```
$ sudo yum install -y package
```

更新インストールの場合：

```
$ sudo yum clean metadata
$ sudo yum update -y package
```

# まとめ

最小構成Yumリポジトリを作ってYumリポジトリ作成手順を学びました。